In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from glob import glob
import IPython.display as ipd
from tqdm.notebook import tqdm
import pandas as pd
import os
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
import random


In [2]:
# path="test"
# folders=os.listdir(path)
# files=[]
# for folder in folders:
#     folder_path = os.path.join(path, folder)
#     folder_files = os.listdir(folder_path)
#     files.extend([(folder, file) for file in folder_files])
    
# df = pd.DataFrame(files, columns=['Folder', 'File'])
# df.to_csv("testfiles.csv",index=False)

In [3]:
# import os
# import shutil
# import random

# def split_videos(src_folder, train_folder, test_folder, split_ratio=0.8):
    
#     os.makedirs(train_folder, exist_ok=True)
#     os.makedirs(test_folder, exist_ok=True)

    
#     subfolders = [f for f in os.listdir(src_folder) if os.path.isdir(os.path.join(src_folder, f))]
    
#     for sub in subfolders:

#         files=os.listdir(f"action/{sub}")
#         train_subfolder_path = os.path.join(train_folder, sub)
#         test_subfolder_path = os.path.join(test_folder, sub)
#         os.makedirs(train_subfolder_path, exist_ok=True)
#         os.makedirs(test_subfolder_path, exist_ok=True)
#         num_train_videos = int(len(files) * split_ratio)
        
#         train_videos = files[:num_train_videos]
#         test_videos = files[num_train_videos:]
#         for video in train_videos:
#             shutil.copy(f"action/{sub}/{video}", f"train/{sub}/{video}")

#         for video in test_videos:
#             shutil.copy(f"action/{sub}/{video}", f"test/{sub}/{video}")

    

# # Replace 'action', 'train', and 'test' with your actual folder paths
# split_videos('action', 'train', 'test')


In [4]:
IMG_SIZE = 256
transforms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])
def read_vid(path):
    images=[]
    cap=cv2.VideoCapture(path)
    ret, img= cap.read()
    while ret:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img=transforms(img)
        images.append(img)
        ret, img= cap.read()
    return images


In [5]:
def collate_fn(batch):
    videos = torch.stack([img  for item in batch for img in random.sample(item[0], k=16)]) # k= frames count
    labels = [item[1] for item in batch]
    labels = torch.as_tensor(labels)
    return [videos, labels]

class videoDataset(Dataset):
    def __init__(self,root,file,):
        self.root=root
        files=pd.read_csv(file)
        self.labels=files.iloc[:,0]
        self.cls = {label: idx for idx, label in enumerate(self.labels.unique())}
        self.video_names=files.iloc[:,1]
    def __getitem__(self, ind) :
        images=read_vid(f"{self.root}/{self.labels[ind]}/{self.video_names[ind]}")
        label=self.cls[self.labels[ind]]
        return images, label
    def __len__(self):
        return len(self.video_names)
        

In [6]:
trainDataset=videoDataset(root="train",file='trainfiles.csv')
train_dataloader = DataLoader(trainDataset, batch_size=10,collate_fn=collate_fn)
testDataset=videoDataset(root="test",file='testfiles.csv')
test_dataloader = DataLoader(testDataset, batch_size=1,collate_fn=collate_fn)

In [7]:
data, label = trainDataset[1]
data[0]

tensor([[[-0.9137, -0.9059, -0.8902,  ..., -0.8431, -0.7882, -0.7412],
         [-0.9451, -0.9373, -0.9294,  ..., -0.8039, -0.7961, -0.7882],
         [-0.9686, -0.9608, -0.9451,  ..., -0.7882, -0.8196, -0.8588],
         ...,
         [-0.0275, -0.0275, -0.0275,  ..., -0.1216, -0.1373, -0.1373],
         [-0.0275, -0.0275, -0.0275,  ..., -0.1216, -0.1373, -0.1373],
         [-0.0275, -0.0275, -0.0275,  ..., -0.1216, -0.1373, -0.1373]],

        [[-0.8980, -0.8902, -0.8745,  ..., -0.8745, -0.8196, -0.7725],
         [-0.9294, -0.9216, -0.9137,  ..., -0.8353, -0.8275, -0.8196],
         [-0.9529, -0.9451, -0.9294,  ..., -0.8275, -0.8588, -0.8980],
         ...,
         [ 0.2000,  0.2000,  0.2000,  ...,  0.0667,  0.0510,  0.0510],
         [ 0.2000,  0.2000,  0.2000,  ...,  0.0667,  0.0510,  0.0510],
         [ 0.2000,  0.2000,  0.2000,  ...,  0.0667,  0.0510,  0.0510]],

        [[-0.9216, -0.9137, -0.8980,  ..., -0.9529, -0.8980, -0.8510],
         [-0.9529, -0.9451, -0.9373,  ..., -0

In [8]:
import torchvision.models as models
import torch
import torch.optim as optim
import torch.nn as nn

In [9]:



cnn_model = models.resnet34(weights='ResNet34_Weights.DEFAULT') 
class AvgCNNModel(nn.Module):
    def __init__(self, cnn_model, output_size=101, frames_cnt=16):
        super(self.__class__, self).__init__()
        self.output_size = output_size
        self.frames_cnt = frames_cnt
        self.cnn = cnn_model
        self.cnn.fc = nn.Linear(self.cnn.fc.in_features, output_size, bias=True)

    def forward(self, videos):
        out = self.cnn(videos)
        out = torch.mean(out.reshape(-1, self.frames_cnt, self.output_size), dim=1)
        return out
    
model = AvgCNNModel(cnn_model, frames_cnt=16)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(10):
    model.train()

    train_loss = []
    for inputs, labels in train_dataloader:

        print(inputs.shape)
        pred_labels = model(inputs)
        loss = criterion(pred_labels, labels)
        train_loss.append(loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Size([160, 3, 256, 256])
torch.Si

In [ ]:
def predict(model, video):
    model.eval()
    inputs, _ = collate_fn([[video, torch.tensor([0])]])
    videos = inputs

    with torch.no_grad():
        pred_labels = model(videos)
    prediction = pred_labels.numpy().argmax()
    return prediction

In [ ]:
test_predictions, test_labels = predict(model, test_dataloader)